# 질문 

GoogLeNet parameter 계산에 대해서 이해가 안가는 부분이 있어서 추가로 글을 쓰게 되었습니다.

우선 첨부파일에서 보시는 바와 같이 첫 번째 convolution layer의 parameter 수가 2.7k라고 나와있습니다. 하지만 직접 계산해본 결과 input size가 224 * 224 * 3일 때 parameter 수를 계산하면 7 * 7 * 3 * 64 = 9408으로 다르게 나옵니다. 제가 무엇을 놓쳤는지 도저히 모르겠어서 질문을 올립니다.

![](https://user-images.githubusercontent.com/38934308/132102522-e372c4d1-87e3-47c5-824b-afdf5a796806.png)  


# 답변

GoogLeNet의 첫번째 레이어는 7x7 convolution입니다.  
입력이 컬러이미지 이므로 채널수는 3입니다. (224x224x**3**)  
출력의 채널수는 64개로 위의 표에서 볼 수 있듯이 64입니다. (112x112x**64**)  
그렇다면 첫번째 레이어의 parameter는 $(7\times7)\times3\times64+64=9472$가 되어야 합니다.

표에 나온 2.7K는 어떻게 된 것일까요?  

동일한 질문의 사례를 보겠습니다.  

[How to calculate the Number of parameters for GoogLe Net?](https://stackoverflow.com/questions/30585122/how-to-calculate-the-number-of-parameters-for-google-net/47880856)  

다른 사람들도 같은 의문을 가지고 있었다는 것을 알 수 있습니다.  

먼저 GoogLeNet의 논문에는 특별한 언급이 없습니다.  

논문링크:  [Going Deeper with Convolutions](https://arxiv.org/abs/1409.4842v1)  

위 링크에 공식 코드가 올려져 있습니다. (Caffe 코드인 것 같습니다.)  

코드(https://worksheets.codalab.org/bundles/0x36208f78facf4a51aee2597bbea901d0) 중에서 `deploy.prototxt`를 보면 GoogLeNet의 Caffe 모델 구조를 볼 수 있습니다.

```
name: "GoogleNet"
input: "data"
input_dim: 10
input_dim: 3
input_dim: 224
input_dim: 224
layer {
  name: "conv1/7x7_s2"
  type: "Convolution"
  bottom: "data"
  top: "conv1/7x7_s2"
  param {
    lr_mult: 1
    decay_mult: 1
  }
  param {
    lr_mult: 2
    decay_mult: 0
  }
  convolution_param {
    num_output: 64
    pad: 3
    kernel_size: 7
    stride: 2
    weight_filler {
      type: "xavier"
      std: 0.1
    }
    bias_filler {
      type: "constant"
      value: 0.2
    }
  }
}
layer {
  name: "conv1/relu_7x7"
  type: "ReLU"
  bottom: "conv1/7x7_s2"
  top: "conv1/7x7_s2"
}
```

논문 내용대로 7x7 convolution (stride=2, padding=3)이고, ReLU activation을 사용합니다.

caffe docker를 이용하여 모델 parameter를 확인해 보았습니다. 

```py
import sys
import os
import caffe
caffe.set_mode_cpu()
import numpy as np
from numpy import prod, sum
from pprint import pprint

def print_net_parameters (deploy_file):
    print "Net: " + deploy_file
    net = caffe.Net(deploy_file, caffe.TEST)

    print "Layer-wise parameters: "
    pprint([(k, v[0].data.shape) for k, v in net.params.items()])
    num = sum( [prod(v[layer].data.shape) for k, v in net.params.items() for layer in range(len(v))] )
    print ("Total number of parameters: {0:,} ".format(num))

#deploy_file = 'deploy.prototxt' 
deploy_file = 'layer1.prototxt'

print_net_parameters(deploy_file)
```
출력:  
```
Layer-wise parameters: 
[('conv1/7x7_s2', (64, 3, 7, 7))]
Total number of parameters: 9,472
```
공식 모델에서도 정확하게 9472개의 parameter가 사용되었다는 것을 확인하였습니다.

아마도 간단한 계산식을 작성하면서 `(7 * 7)*3*64+64`를 `(7 + 7)*3*64+64`로 곱하기가 더하기로 오타나면 논문의 parameter 수인 2.7K과 같아지는데 이렇게 된 것이 아닐까 추측해 봅니다.  
stackoverflow 답변중에 (7x7) convolution을 (7x1),(1x7)로 사용하면 parameter수가 같아진다는 내용이 있는데, 공식 코드를 보면 그것은 아닌 것 같습니다.  

Inception V3의 논문을 보면, (7x7)보다 (7x1)+(1x7)이 더 낫다는 얘기가 나옵니다. 관심있으신 분들은 읽어보시면 좋을 것 같습니다.  

[Rethinking the Inception Architecture for Computer Vision](https://arxiv.org/abs/1512.00567v3)  




## 참고 keras 코드  (GoogLeNet)

다음의 커뮤니티 코드를 참조하였습니다.

https://github.com/jhcha08/Implementation_DeepLearningPaper/blob/master/CNN.%20GoogleNet.ipynb

In [ ]:

from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, Dense, Dropout, AveragePooling2D, concatenate, Flatten
from keras.models import Model
from keras import models

def inception(x, c_1, c_2a, c_2b, c_3a, c_3b, c_m):
    
    pre_layer = x
    
    conv1 = Conv2D(filters = c_1, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(pre_layer)
    
    conv2 = Conv2D(filters = c_2a, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(pre_layer)
    conv2 = Conv2D(filters = c_2b, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu')(conv2)
    
    conv3 = Conv2D(filters = c_3a, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(pre_layer)
    conv3 = Conv2D(filters = c_3b, kernel_size = (5,5), strides = (1,1), padding = 'same', activation = 'relu')(conv3)
    
    maxpool = MaxPooling2D(pool_size = (3,3), strides = (1,1), padding = 'same')(pre_layer)
    maxconv = Conv2D(filters = c_m, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(maxpool)
    
    concat = concatenate([conv1, conv2, conv3, maxconv], axis = -1) # [B,W,H,C] 형태로 결과가 나오므로 채널인 'C' 기준으로 concat
    
    return concat

inputs = Input(shape=(224,224,3))

x = Conv2D(filters = 64, kernel_size = (7,7), strides = (2,2), padding = 'same', activation = 'relu')(inputs)
x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = Conv2D(filters = 64, kernel_size = (1,1), strides = (1,1), padding = 'valid', activation = 'relu')(x)
x = Conv2D(filters = 192, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = inception(x, 64, 96, 128, 16, 32, 32) # 3a
x = inception(x, 128, 128, 192, 32, 96, 64) # 3b

x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = inception(x, 192, 96, 208, 16, 48, 64) # 4a
x = inception(x, 160, 112, 224, 24, 64, 64) # 4b

aux0 = AveragePooling2D(pool_size = (5,5), strides = (3,3), padding = 'valid')(x)
aux0 = Conv2D(filters = 128, kernel_size = (1,1), strides = (3,3), padding = 'same')(aux0)
aux0 = Dense(1024, activation = 'relu')(aux0)
aux0 = Dense(1024, activation = 'relu')(aux0)
softmax0 = Dense(1000, activation = 'softmax')(aux0)

x = inception(x, 128, 128, 256, 24, 64, 64) # 4c
x = inception(x, 112, 144, 288, 32, 64, 64) # 4d
x = inception(x, 256, 160, 320, 32, 128, 128) # 4e

aux1 = AveragePooling2D(pool_size = (5,5), strides = (3,3), padding = 'valid')(x)
aux1 = Conv2D(filters = 128, kernel_size = (1,1), strides = (3,3), padding = 'same')(aux1)
aux1 = Dense(1024, activation = 'relu')(aux1)
aux1 = Dense(1024, activation = 'relu')(aux1)
softmax1 = Dense(1000, activation = 'softmax')(aux1)

x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = inception(x, 256, 160, 320, 32, 128, 128) # 5a
x = inception(x, 384, 192, 384, 48, 128, 128) # 5b

x = AveragePooling2D(pool_size=(7,7), strides = (1,1), padding = 'valid')(x)
x = Dropout(0.4)(x)
x = Flatten()(x)

outputs = Dense(1000, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = [aux0, aux1, outputs])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 64)   4160        max_pooling2d[0][0]              
____________________________________________________________________________________________